# Publish a Training Pipeline
In this notebook, we will show how to automate the training/retraining of model using HyperDrive and registering best model. Once this training pipeline is published/created, it provides a REST endpoint which can be called to run this pipeline without using the Azure Machine Learning Service SDK.


## Imports  

In [ ]:
import os
import requests

import azureml
from azureml.core import Workspace, Experiment
from azureml.core.datastore import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import HyperDriveStep, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.core.runconfig import RunConfiguration, CondaDependencies
from azureml.train.dnn import PyTorch
from azureml.core.container_registry import ContainerRegistry
from azureml.train.hyperdrive import (
    RandomParameterSampling,
    BanditPolicy,
    uniform,
    choice,
    HyperDriveConfig,
    PrimaryMetricGoal,
)
from azureml.widgets import RunDetails

from dotenv import set_key, get_key, find_dotenv
from utilities import get_auth

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

from MetricsUtils.hpStatisticsCollection import statisticsCollector, CollectionEntry
from MetricsUtils.storageutils import storageConnection

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Read in the Azure ML workspace and default datastore
Read in the the workspace created in a previous notebook.

In [ ]:
auth = get_auth(env_path)
ws = Workspace.from_config(auth=auth)
print(ws.name, ws.resource_group, ws.location, sep="\n")

In [ ]:
ds = ws.get_default_datastore()

## Tune Model Hyperparameters
We automatically tune hyperparameters by exploring the range of values defined for each hyperparameter. Here we use random sampling which randomly selects hyperparameter values from the defined search space. Random sampling allows for both discrete and continuous hyperparameters.

In [ ]:
param_sampling = RandomParameterSampling(
    {
        "learning_rate": uniform(0.0005, 0.005),
        "rpn_nms_thresh": uniform(0.3, 0.7),
        "anchor_sizes": choice(
            "16",
            "16,32",
            "16,32,64",
            "16,32,64,128",
            "16,32,64,128,256",
            "16,32,64,128,256,512",
        ),
        "anchor_aspect_ratios": choice(
            "0.25", "0.25,0.5", "0.25,0.5,1.0", "0.25,0.5,1.0,2.0"
        ),
    }
)

The num epochs and maximum total run parameters deliberately have a low default value for the speed of running. In actual application, set these to higher values (i.e. num_epochs = 10, max_total_runs = 16)

In [ ]:
# number of epochs
num_epochs = 1

# max total runs for hyperdrive
max_total_runs = 1

It is also possible to specify a maximum duration for the tuning experiment by setting `max_duration_minutes`. If both of these parameters are specified, any remaining runs are terminated once `max_duration_minutes` have passed.

We will terminate poorly performing runs automatically with bandit early termination policy which is based on slack factor and evaluation interval. The policy terminates any run where the primary metric is not within the specified slack factor with respect to the best performing training run.

In [ ]:
early_termination_policy = BanditPolicy(
    slack_factor=0.15, evaluation_interval=2, delay_evaluation=2
)

## Create an estimator <a id='estimator'></a>
Create an estimator that specifies the location of the script, sets up its fixed parameters, including the location of the data, the compute target, and specifies the packages needed to run the script. It may take a while to prepare the run environment the first time an estimator is used, but that environment will be used until the list of packages is changed.

In [ ]:
cluster_name = get_key(env_path, 'cluster_name')

In [ ]:
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC6", max_nodes=8
    )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster.
print(compute_target.get_status().serialize())

In [ ]:
script_folder = "./torchdetect"
image_name = get_key(env_path, "image_name")

In [ ]:
# point to an image in private ACR
image_registry_details = ContainerRegistry()
image_registry_details.address = get_key(env_path, "acr_server_name")
image_registry_details.username = get_key(env_path, "acr_username")
image_registry_details.password = get_key(env_path, "acr_password")

In [ ]:
estimator = PyTorch(
    source_directory=script_folder,
    compute_target=compute_target,
    entry_script="train.py",
    use_docker=True,
    custom_docker_image=image_name,
    image_registry_details=image_registry_details,
    user_managed=True,
    use_gpu=True,
)

estimator.run_config.environment.environment_variables["PYTHONPATH"] = "$PYTHONPATH:/cocoapi/PythonAPI/"

Put the estimator and the configuration information together into an HyperDrive run configuration object.

In [ ]:
hyperdrive_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="mAP@IoU=0.50",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=max_total_runs,
    max_concurrent_runs=4,
)

## Azure Machine Learning Pipelines: Overview <a id='aml_pipeline_overview'></a>

A common scenario when using machine learning components is to have a data workflow that includes the following steps:

- Preparing/preprocessing a given dataset for training, followed by
- Training a machine learning model on this data, and then
- Deploying this trained model in a separate environment, and finally
- Running a batch scoring task on another data set, using the trained model.

Azure's Machine Learning pipelines give you a way to combine multiple steps like these into one configurable workflow, so that multiple agents/users can share and/or reuse this workflow. Machine learning pipelines thus provide a consistent, reproducible mechanism for building, evaluating, deploying, and running ML systems.

To get more information about Azure machine learning pipelines, please read our [Azure Machine Learning Pipelines overview](https://aka.ms/pl-concept), or the [getting started notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-getting-started.ipynb).

Let's create a data reference for the raw data to be used in HyperDrive run.

In [ ]:
data_folder = DataReference(datastore=ds, data_reference_name="data_folder")

In [ ]:
model_name = PipelineParameter(name="model_name", default_value="torchvision_best_model")

## Create AML Pipeline Tuning Step 
We create a HyperDrive step in the AML pipeline to perform a search for hyperparameters. The `tune_epochs` pipeline parameter that controls the number of epochs used in tuning deliberately has a low default value for the speed of pipeline testing. 

In [ ]:
tune_step_name="tune_model"
tune_epochs = PipelineParameter(name="tune_epochs", default_value=1)  # Set to 10 when running the pipeline.

In [ ]:
tune_step = HyperDriveStep(
    name=tune_step_name,
    hyperdrive_config=hyperdrive_config,
    estimator_entry_script_arguments=["--data_path", data_folder,
                                      "--workers", 8,
                                      "--epochs", tune_epochs,
                                      "--box_nms_thresh", 0.3,
                                      "--box_score_thresh", 0.10],
    inputs=[data_folder],
    allow_reuse=False)

## Create AML Pipeline Register Model Step
This Python script step registers the best model found by the HyperDrive step.

Let's create a folder for the script.

In [ ]:
script_folder = "./registermodel"
os.makedirs(script_folder, exist_ok=True)

In [ ]:
%%writefile registermodel/Register_Model.py

from __future__ import print_function
import os
import json
import argparse

from azureml.core import Run
from azureml.pipeline.core import PipelineRun
from azureml.pipeline.steps import HyperDriveStepRun
import azureml.core

if __name__ == "__main__":
    
    print("azureml.core.VERSION={}".format(azureml.core.VERSION))

    parser = argparse.ArgumentParser(description="Register the model created by"
                                     " an HyperDrive step")
    parser.add_argument("--hd-step", dest="hd_step",
                        help="the name of the HyperDrive step")
    parser.add_argument("--outputs", help="the model file outputs directory")
    parser.add_argument("--model-name", dest="model_name",
                        help="the model file base name")
    args = parser.parse_args()
    
    model_name = args.model_name
    model_file = "model_latest.pth"
    model_path = os.path.join(args.outputs, model_file)
    
    # Get the HyperDrive run.
    run = Run.get_context()
    print(run)
    pipeline_run = PipelineRun(run.experiment, run.parent.id)
    print(pipeline_run)
    hd_step_run = HyperDriveStepRun(step_run=pipeline_run.find_step_run(args.hd_step)[0])
    print(hd_step_run)
    
    # Get the best run.
    hd_step_run.wait_for_completion(show_output=True, wait_post_processing=True)
    best_run = hd_step_run.get_best_run_by_primary_metric()   
    if best_run is None:
        raise Exception("No best run was found")
    print(best_run)
    
    # Register the model
    model = best_run.register_model(model_name=model_name, model_path=model_path)
    print("Best Model registered")

Creating PythonScript Step for AML pipeline to register the best model. The `bm_steps_data` input pipeline data is only used to synchronize with the previous pipeline step.

In [ ]:
rm_step_name = "register_model"
rm_run_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
    pip_packages=["azure-cli", "azureml-sdk", "azureml-pipeline"]))
rm_run_config.environment.docker.enabled = True
rm_step = PythonScriptStep(
    name=rm_step_name,
    script_name="Register_Model.py",
    compute_target=compute_target,
    source_directory=os.path.join(".", "registermodel"),
    arguments=["--hd-step", tune_step_name,
               "--outputs", "outputs",
               "--model-name", model_name],
    runconfig=rm_run_config,
    allow_reuse=False)

Let's specify to run register model step after tune model step in the pipeline.

In [ ]:
rm_step.run_after(tune_step)

## Create & Run a Pipeline
When we specify the rm_step, Pipeline walks the dependency graph to include the other steps.

In [ ]:
experiment_name = "torchvision"
exp = Experiment(workspace=ws, name=experiment_name)
pipeline = Pipeline(workspace=ws, steps=[rm_step])
pipeline.validate()

Run the pipeline before publishing it. Wait for the run to complete.

In [ ]:
pipeline_run = exp.submit(pipeline, continue_on_step_failure=True)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion(show_output=True)

## Publish The Pipeline 
You may read more about why to publish a pipeline and how the published pipeline can be triggered [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines)

In [ ]:
published_pipeline = pipeline.publish(name="DL HyperDrive Pipeline",
                                      description="DL HyperDrive Pipeline",
                                      continue_on_step_failure=True)
published_pipeline.endpoint

## Run published pipeline using its REST endpoint
This step shows how to call the REST endpoint of a published pipeline to trigger the pipeline run. 

In [ ]:
# aad_token = auth.get_authentication_header()
# rest_endpoint = published_pipeline.endpoint
# print("You can perform HTTP POST on URL {} to trigger this pipeline".format(rest_endpoint))
# response = requests.post(rest_endpoint, 
#                          headers=aad_token, 
#                          json={"ExperimentName": experiment_name,
#                                "RunSource": "SDK"})
# run_id = response.json()["Id"]
# print(run_id)

## Schedule the published pipeline to run regularly
This step shows how to schedule the published pipeline to run regularly. This will also submit an initial run since a starting time for the schedule is not supplied.

In [ ]:
schedule_name = "HypeTuningDL"
frequency = "Hour"
interval = 2
recurrence = ScheduleRecurrence(frequency=frequency, interval=interval)
schedule = Schedule.create(
    workspace=ws,
    name=schedule_name,
    pipeline_id=published_pipeline.id,
    experiment_name=experiment_name,
    recurrence=recurrence,
    description=schedule_name)

## Write Out the URI 
Write the URI to the statistics tracker.

In [ ]:
statisticsCollector.addEntry(CollectionEntry.AKS_REALTIME_ENDPOINT,
                             published_pipeline.endpoint)

Get a connection string to the workspace's storage to use to save the statistics.

In [ ]:
storageConnString = "YOUR_STORAGE_CONNECTION_STRING"

In [ ]:
if storageConnString == "YOUR_STORAGE_CONNECTION_STRING":
    resource_group = ws.resource_group
    stgAcctName = ws.get_details()['storageAccount'].split('/')[-1]
    storageConnString = storageConnection.getConnectionStringWithAzCredentials(resource_group, stgAcctName)
print("storage_conn_string: {}".format(storageConnString))

Save the statistics collected so far.

In [ ]:
statisticsCollector.uploadContent(storageConnString)

You can now proceed to the next notebook to [delete the resources of this tutorial](06_TearDown.ipynb).